# Numerical Methods Final Project

### Team Members
1) Pinal Gajjar <br>
2) Dharani Goalla <br>
3) Venkata Narasa Reddy Boreddy <br>

## Task 1: Implement the simple logistic regression model

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Sigmoid function for logistic regression
def sigmoid(z):
    # Clip z to prevent overflow.
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

# Logistic regression probability function
def logistic_regression_prob(X, theta):
    return sigmoid(np.dot(X, theta))

# Log likelihood for a simple logistic regression
def log_likelihood_simple(theta, X, y):
    predictions = logistic_regression_prob(X, theta)
    epsilon = 1e-5  # Avoid log(0)
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    log_likelihood_value = np.sum(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    return -log_likelihood_value  # Minimize negative log likelihood

# Gradient for a simple logistic regression
def log_likelihood_gradient_simple(theta, X, y):
    predictions = logistic_regression_prob(X, theta)
    errors = y - predictions
    gradient = np.dot(X.T, errors)
    return gradient

# Gradient ascent optimization for simple logistic regression
def gradient_ascent_simple(X, y, learning_rate, iterations):
    theta = np.zeros(X.shape[1])
    for i in range(iterations):
        gradient = log_likelihood_gradient_simple(theta, X, y)
        theta += learning_rate * gradient
        if i % 100 == 0:  # Print log likelihood every 100 iterations
            print(f'Log Likelihood at iteration {i}: {log_likelihood_simple(theta, X, y)}')
    return theta

# Load and prepare your data
wine_data_path = 'https://raw.githubusercontent.com/PinalG/Dynamic-ensemble-logistic-regression-model/main/allwine.csv'  # Replace with the correct path
wine_data = pd.read_csv(wine_data_path)

# Prepare the data
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']
X = np.hstack([np.ones((X.shape[0], 1)), X])  # Add intercept term

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# TASK 1: Simple Logistic Regression
learning_rate = 0.01
iterations = 5000
theta_simple = gradient_ascent_simple(X_train, y_train, learning_rate, iterations)

# Make predictions using the simple logistic regression model
predictions_simple = logistic_regression_prob(X_test, theta_simple) >= 0.5

# Calculate accuracy for the simple logistic regression model
accuracy_simple = accuracy_score(y_test, predictions_simple)
print(f'Accuracy of the simple logistic regression model: {accuracy_simple}')


Log Likelihood at iteration 0: 4522.761948452693
Log Likelihood at iteration 100: 7062.677524559342
Log Likelihood at iteration 200: 7044.520241800876
Log Likelihood at iteration 300: 7044.658450518149
Log Likelihood at iteration 400: 7044.657756027635
Log Likelihood at iteration 500: 7044.657758614851
Log Likelihood at iteration 600: 7044.6577586232215
Log Likelihood at iteration 700: 7044.657758623096
Log Likelihood at iteration 800: 7044.657758623096
Log Likelihood at iteration 900: 7044.657758623096
Log Likelihood at iteration 1000: 7044.657758623096
Log Likelihood at iteration 1100: 7044.657758623096
Log Likelihood at iteration 1200: 7044.657758623096
Log Likelihood at iteration 1300: 7044.657758623096
Log Likelihood at iteration 1400: 7044.657758623096
Log Likelihood at iteration 1500: 7044.657758623096
Log Likelihood at iteration 1600: 7044.657758623096
Log Likelihood at iteration 1700: 7044.657758623096
Log Likelihood at iteration 1800: 7044.657758623096
Log Likelihood at itera

## Task 2: Dynamic Ensemble Logistic Regression Model

In [2]:
def log_likelihood_gradient(theta, X, y):
    theta_M = theta[:X.shape[1]]
    theta_L = theta[X.shape[1]:2*X.shape[1]]
    theta_R = theta[2*X.shape[1]:]

    h_thetaM = logistic_regression_prob(X, theta_M)
    h_thetaL = logistic_regression_prob(X, theta_L)
    h_thetaR = logistic_regression_prob(X, theta_R)

    common_term = y - (h_thetaL * h_thetaM + h_thetaR * (1 - h_thetaM))
    grad_M = np.dot(X.T, common_term * (h_thetaL - h_thetaR))
    grad_L = np.dot(X.T, common_term * h_thetaM)
    grad_R = np.dot(X.T, -common_term * (1 - h_thetaM))

    # Combine gradients
    grad = np.concatenate([grad_M, grad_L, grad_R])

    return grad

# Probability function for the ensemble model
def probability_function(X, theta_M, theta_L, theta_R, y):
    h_thetaM = logistic_regression_prob(X, theta_M)
    h_thetaL = logistic_regression_prob(X, theta_L)
    h_thetaR = logistic_regression_prob(X, theta_R)

    P_1 = h_thetaL * h_thetaM + h_thetaR * (1 - h_thetaM)
    P_0 = (1 - h_thetaL) * h_thetaM + (1 - h_thetaR) * (1 - h_thetaM)

    return np.where(y == 1, P_1, P_0)

# Gradient ascent for logistic regression
def logistic_regression_gradient_ascent(X, y, theta, learning_rate, iterations):
    for i in range(iterations):
        grad = logistic_regression_gradient(theta, X, y)
        theta += learning_rate * grad
    return theta

# Gradient ascent for the ensemble model
def gradient_ascent_ensemble(X, y, theta, learning_rate, iterations):
    for i in range(iterations):
        grad = log_likelihood_gradient(theta, X, y)
        theta += learning_rate * grad
        # if i % 100 == 0:  # Print log likelihood every 100 iterations
        #     print(f'Log Likelihood at iteration {i}: {log_likelihood_ensemble(theta, X, y)}')
    return theta


theta_initial = np.zeros(X_train.shape[1] * 3)

# Hyperparameters for gradient ascent
learning_rate = 0.0028  # tune learning rates
iterations = 10000  # tune number of iterations

# Run gradient ascent to maximize the log likelihood for the ensemble model
theta_optimal_ensemble = gradient_ascent_ensemble(X_train, y_train, theta_initial, learning_rate, iterations)

# Extract the optimized parameters for each model
theta_M_ensemble = theta_optimal_ensemble[:X_train.shape[1]]
theta_L_ensemble = theta_optimal_ensemble[X_train.shape[1]:2*X_train.shape[1]]
theta_R_ensemble = theta_optimal_ensemble[2*X_train.shape[1]:]

# Make predictions using the ensemble model
ensemble_probs_task2 = probability_function(X_test, theta_M_ensemble, theta_L_ensemble, theta_R_ensemble, y_test)
ensemble_preds_task2 = ensemble_probs_task2 >= 0.5

# Calculate accuracy for the ensemble model
accuracy_ensemble_task2 = accuracy_score(y_test, ensemble_preds_task2)
print(f'Accuracy of the ensemble model: {accuracy_ensemble_task2}')

Accuracy of the ensemble model: 0.8125


## Task 3: Deep Dynamic Ensemble Model Optimization


In [3]:
# Gradient of log likelihood for the three-layer ensemble model
def gradient_ensemble(thetas, X, y, lambda_reg=0.01):
    theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, theta_7 = np.split(thetas, 7)
    p_1 = logistic_regression_prob(X, theta_1)
    p_2 = logistic_regression_prob(X, theta_2)
    p_3 = logistic_regression_prob(X, theta_3)
    p_4 = logistic_regression_prob(X, theta_4)
    p_5 = logistic_regression_prob(X, theta_5)
    p_6 = logistic_regression_prob(X, theta_6)
    p_7 = logistic_regression_prob(X, theta_7)

    final_prediction = (p_4 * p_2 * p_1 +
                        p_5 * (1 - p_2) * p_1 +
                        p_6 * p_3 * (1 - p_1) +
                        p_7 * (1 - p_3) * (1 - p_1))
    final_prediction = np.clip(final_prediction, 1e-10, 1 - 1e-10)

    error = y - final_prediction
    grad_theta_1 = np.dot(X.T, error * ((p_2 * p_4 + (1 - p_2) * p_5) - (p_3 * p_6 + (1 - p_3) * p_7))) + lambda_reg * theta_1
    grad_theta_2 = np.dot(X.T, error * p_1 * (p_4 - p_5)) + lambda_reg * theta_2
    grad_theta_3 = np.dot(X.T, error * (1 - p_1) * (p_6 - p_7)) + lambda_reg * theta_3
    grad_theta_4 = np.dot(X.T, error * p_2 * p_1) + lambda_reg * theta_4
    grad_theta_5 = np.dot(X.T, error * (1 - p_2) * p_1) + lambda_reg * theta_5
    grad_theta_6 = np.dot(X.T, error * p_3 * (1 - p_1)) + lambda_reg * theta_6
    grad_theta_7 = np.dot(X.T, error * (1 - p_3) * (1 - p_1)) + lambda_reg * theta_7

    gradients = np.concatenate([grad_theta_1, grad_theta_2, grad_theta_3,
                                grad_theta_4, grad_theta_5, grad_theta_6, grad_theta_7])
    return gradients

# Gradient ascent optimization function
def gradient_ascent(X, y, learning_rate=0.00005, iterations=20000, gradient_function=gradient_ensemble, lambda_reg=1):
    thetas = np.zeros(X.shape[1] * 7)  # Initialize parameters for all nodes
    for i in range(iterations):
        gradient = gradient_function(thetas, X, y, lambda_reg)
        thetas += learning_rate * gradient

        if i % 1000 == 0:  # Print log likelihood every 1000 iterations
            # Compute the final prediction for log likelihood
            theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, theta_7 = np.split(thetas, 7)
            p_1 = logistic_regression_prob(X, theta_1)
            p_2 = logistic_regression_prob(X, theta_2)
            p_3 = logistic_regression_prob(X, theta_3)
            p_4 = logistic_regression_prob(X, theta_4)
            p_5 = logistic_regression_prob(X, theta_5)
            p_6 = logistic_regression_prob(X, theta_6)
            p_7 = logistic_regression_prob(X, theta_7)

            final_prediction = (p_4 * p_2 * p_1 +
                                p_5 * (1 - p_2) * p_1 +
                                p_6 * p_3 * (1 - p_1) +
                                p_7 * (1 - p_3) * (1 - p_1))
            final_prediction = np.clip(final_prediction, 1e-10, 1 - 1e-10)

            ll = -np.mean(y * np.log(final_prediction) + (1 - y) * np.log(1 - final_prediction))
            print(f'Iteration {i}, Log Likelihood: {ll}')

    return thetas

learning_rate_task3 = 0.0001  #   tune this
iterations_task3 = 10000     #  tune this

# Optimize the deep ensemble model
thetas_optimized = gradient_ascent(
    X_train, y_train,
    learning_rate_task3, iterations_task3,
    gradient_ensemble
)

# Extract the optimized parameters for each node
theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, theta_7 = np.split(thetas_optimized, 7)
def compute_deep_ensemble_probs(X, theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, theta_7):
    p_1 = logistic_regression_prob(X, theta_1)
    p_2 = logistic_regression_prob(X, theta_2)
    p_3 = logistic_regression_prob(X, theta_3)
    p_4 = logistic_regression_prob(X, theta_4)
    p_5 = logistic_regression_prob(X, theta_5)
    p_6 = logistic_regression_prob(X, theta_6)
    p_7 = logistic_regression_prob(X, theta_7)

    final_prediction = (p_4 * p_2 * p_1 +
                        p_5 * (1 - p_2) * p_1 +
                        p_6 * p_3 * (1 - p_1) +
                        p_7 * (1 - p_3) * (1 - p_1))
    final_prediction = np.clip(final_prediction, 1e-10, 1 - 1e-10)
    return final_prediction

# Make predictions using the deep ensemble model
deep_ensemble_probs_task3 = compute_deep_ensemble_probs(X_test, theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, theta_7)
deep_ensemble_preds_task3 = deep_ensemble_probs_task3 >= 0.5

# Calculate accuracy for the deep ensemble model
accuracy_deep_ensemble_task3 = accuracy_score(y_test, deep_ensemble_preds_task3)
print(f'Accuracy of the deep ensemble model: {accuracy_deep_ensemble_task3}')

Iteration 0, Log Likelihood: 0.6875286323505992
Iteration 1000, Log Likelihood: 0.5343642298977115
Iteration 2000, Log Likelihood: 0.5339710835519595
Iteration 3000, Log Likelihood: 0.5339349564410933
Iteration 4000, Log Likelihood: 0.5339483107905347
Iteration 5000, Log Likelihood: 0.5339653465958092
Iteration 6000, Log Likelihood: 0.5339774407885458
Iteration 7000, Log Likelihood: 0.533984793507454
Iteration 8000, Log Likelihood: 0.5339889871570344
Iteration 9000, Log Likelihood: 0.533991307773959
Accuracy of the deep ensemble model: 0.7484375


## Task 4 Generalizing the model to support arbitrary number of layers


In [4]:
# Logistic Regression Probability Function
def logistic_regression_prob(X, theta):
    return 1 / (1 + np.exp(-np.dot(X, theta)))

# Generalized Gradient of Log Likelihood for Ensemble Model
def gradient_ensemble(thetas, X, y, num_layers, lambda_reg=0.01):
    thetas_split = np.split(thetas, num_layers)
    probs = [logistic_regression_prob(X, theta) for theta in thetas_split]

    final_prediction = probs[0]
    for i in range(1, num_layers):
        final_prediction = final_prediction * probs[i] + (1 - final_prediction) * probs[i]
    final_prediction = np.clip(final_prediction, 1e-10, 1 - 1e-10)

    gradients = []
    for i in range(num_layers):
        error = y - final_prediction
        partial_grad = np.dot(X.T, error * final_prediction * (1 - final_prediction)) + lambda_reg * thetas_split[i]
        gradients.append(partial_grad)

    return np.concatenate(gradients)

# Gradient Ascent Optimization Function
def gradient_ascent(X, y, num_layers, learning_rate=0.00005, iterations=20000, lambda_reg=1):
    thetas = np.zeros(X.shape[1] * num_layers)
    for i in range(iterations):
        gradient = gradient_ensemble(thetas, X, y, num_layers, lambda_reg)
        thetas += learning_rate * gradient
        # Optional: Add log likelihood computation here
    return thetas

# Generalized Ensemble Prediction Function
def predict_ensemble(X, thetas, num_layers):
    thetas_split = np.split(thetas, num_layers)
    final_prediction = logistic_regression_prob(X, thetas_split[0])
    for i in range(1, num_layers):
        final_prediction = final_prediction * logistic_regression_prob(X, thetas_split[i]) + (1 - final_prediction) * logistic_regression_prob(X, thetas_split[i])
    return final_prediction

# Load dataset
wine_data = pd.read_csv('https://raw.githubusercontent.com/PinalG/Dynamic-ensemble-logistic-regression-model/main/allwine.csv')

# Prepare data
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Set hyperparameters
num_layers = 7  # Experiment with this value
learning_rate = 0.001  # Adjust learning rate
iterations = 15000  # Increase iterations
lambda_reg = 0.01  # Experiment with regularization strength

# Train model
thetas_optimized = gradient_ascent(X_train, y_train, num_layers, learning_rate, iterations, lambda_reg)

# Predict and evaluate
ensemble_predictions = predict_ensemble(X_test, thetas_optimized, num_layers)
predictions = np.where(ensemble_predictions >= 0.5, 1, 0)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy of the generalized ensemble model: {accuracy}")


Accuracy of the generalized ensemble model: 0.7421875
